# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

In [166]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [21]:
# Function for commonly calculated stats
def statistics(df):
    perc_passing_math = (df["math_score"]>=70).sum()*100/df["Student ID"].count()
    perc_passing_reading = (df["reading_score"]>=70).sum()*100/df["Student ID"].count()

    return pd.DataFrame(
    {
        "Average Math Score": [df["math_score"].mean()],
        "Average Reading Score": [df["reading_score"].mean()],
        "% Passing Math": [perc_passing_math],
        "% Passing Reading": [perc_passing_reading],
        "% Overall Passing Rate": [(perc_passing_math+perc_passing_reading)/2]
    })

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [33]:
district_summary = pd.DataFrame(
    {
        "Total Schools": ["{:,}".format(school_data["School ID"].count())],
        "Total Students": ["{:,}".format(school_data["size"].sum())],
        "Total Budget": ["${:,}".format(school_data["budget"].sum())]
    }
    )

district_summary = district_summary.join(statistics(school_data_complete))
district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [34]:
# pull only the data needed for the school summary 
school_summary = school_data.loc[:,["school_name","type","size","budget"]].set_index("school_name")

# calculate the Per Student Budget
school_summary["Per Student Budget"] = school_summary["budget"]/school_summary["size"]

# run statistics on the data grouped by school_name to get the common statistics
school_stats = school_data_complete.groupby("school_name").apply(statistics).reset_index(level=1, drop=True)

# merge the common stats with the school summary stats
school_summary = pd.merge(school_summary, school_stats , left_index=True, right_index=True)

# make it pretty & print
school_summary = school_summary.rename(columns={"type":"School Type","size":"Total Students", "budget": "Total School Budget"})
school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,94.860875
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [14]:
school_summary.sort_values("% Overall Passing Rate", ascending=False).head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,97.039828,94.133477,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,97.308869,93.272171,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,95.945946,94.594595,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,97.138965,93.392371,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,96.539641,93.867718,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [15]:
school_sammary
school_summary.sort_values("% Overall Passing Rate").head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,80.220055,66.366592,73.293323
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,80.739234,65.988471,73.363852
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,81.316421,65.683922,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,81.222432,66.057551,73.639992
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,79.299014,68.309602,73.804308


## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [35]:
# used a pivot table to simplify the steps
math_scores = pd.pivot_table(school_data_complete, columns = "grade", index="school_name", values="math_score", 
                             aggfunc=np.mean)

# make it pretty and print
math_scores = math_scores[['9th','10th','11th','12th']]
math_scores.head()

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [36]:
reading_scores = pd.pivot_table(school_data_complete, columns = "grade", index="school_name", values="reading_score", 
                                aggfunc=np.mean)
reading_scores = reading_scores[['9th','10th','11th','12th']]
reading_scores.head()

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [38]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [39]:
# select the data and find the Per Student Budget
school_spending = school_data_complete.loc[:,["size","budget","reading_score","math_score", "Student ID"]]
school_spending["Per Student Budget"] = school_spending["budget"]/school_spending["size"]

# bin the per student budget into spending ranges
school_spending["Spending Range (Per Student)"] = pd.cut(school_spending["Per Student Budget"], spending_bins, labels=group_names)

# group by school spending and get the stats
school_spending_ranges = school_spending.groupby("Spending Range (Per Student)").apply(statistics).reset_index(level=1, drop=True)
school_spending_ranges.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Range (Per Student),,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


## Scores by School Size

* Perform the same operations as above, based on school size.

In [116]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [117]:
# select the data 
school_size = school_data_complete.loc[:,["size","budget","reading_score","math_score","Student ID"]]

# bin the data into different school size ranges
school_size["School Size"] = pd.cut(school_size["size"], size_bins, labels=group_names)

# group by school size to get the stats
school_size_ranges = school_size.groupby("School Size").apply(statistics).reset_index(level=1, drop=True)
school_size_ranges.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.974082,96.040317,93.952484,94.996400
Medium (1000-2000),83.372682,83.867989,96.773058,93.616522,95.194790
Large (2000-5000),77.477597,81.198674,82.125158,68.652380,75.388769


## Scores by School Type

* Perform the same operations as above, based on school type.

In [45]:
# group by type and apply stats
school_type = school_data_complete.groupby("type").apply(statistics).reset_index(level=1, drop=True)

# pretty print
school_type = school_type.rename_axis("School Type")
school_type.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
